# Method 1 - (USING GQB)

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from getpass import getpass

In [ ]:
#first create a API query using Google Query Builder: https://ga-dev-tools.appspot.com/query-explorer/

def get_ga_report():
    
    api_query_uri = getpass("Enter Your API_URI") #getting user to enter the query
    r = requests.get(api_query_uri) # requesting the data from API
    data = r.json() #storing the data into json
    
    columns = [i.get("name") for i in data["columnHeaders"]]  #reading the headers from the json file
    df = pd.DataFrame(data = data["rows"], columns = columns) #creating a data frame 
    
    df.to_csv("ga_data.csv") #exporting the data into csv
    
    return df #returnnig the dataframe

In [ ]:
final_data = get_ga_report() #creating a variable

Enter Your API_URI··········


In [ ]:
final_data.head(2)

,ga:userType,ga:newUsers,ga:sessions
0,New Visitor,524,524
1,Returning Visitor,0,269


# Method - 2 (USING GOOGLE ANLYTICS REPORTING API.V4)

In [ ]:
# Updated Final Working Code

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

uploaded = files.upload() # uploading the file ti Google Colab
file_n = "credential.json" #saving the filename to the variable

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly'] #saving the API URL 
KEY_FILE_LOCATION = file_n #adding JSON file
VIEW_ID = '141647812'  #adding viewID
DIMENSION = ['ga:source', 'ga:medium'] #passing dimension of GA which needs to be fetch
METRICS = ['ga:sessions', 'ga:users', 'ga:pageviews', 'ga:bounceRate'] #passing the metrics which needs to be fetch

credentials = ServiceAccountCredentials.from_json_keyfile_name(KEY_FILE_LOCATION, SCOPES)
analytics = build('analyticsreporting', 'v4', credentials = credentials)

raw_data = analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{'startDate': '2017-01-01', 'endDate': '2017-12-31'}],
          'metrics': [{'expression': i} for i in METRICS],
          'dimensions': [{'name': j} for j in DIMENSION],
          'orderBys':[
              {'fieldName': 'ga:pageviews', 'sortOrder': 'DESCENDING'}
          ],
          'samplingLevel': "LARGE",
          'includeEmptyRows': True,
          'metricFilterClauses': [{
           'filters':[{
               'metricName': "ga:sessions",
               'operator': "GREATER_THAN",
               'comparisonValue': "0"
           }]
          }] 
        }]
      }
  ).execute()

a = []
b = []
c = []
d = []

for report in raw_data.get("reports", []):
    columnHeader = report.get("columnHeader", {})
    dimensionHeaders = columnHeader.get("dimensions", [])
    metricsHeader = columnHeader.get("metricHeader", {}).get("metricHeaderEntries", [])
    a.append(dimensionHeaders)
    
    for row in report.get('data', {}).get("rows", []):
        dimensions = row.get("dimensions", [])
        metric = row.get("metrics", [])
        
        b.append(dimensions)
        
        for rows in metric:
            d.append(rows.get("values"))
    
    for col in metricsHeader:
        c.append(col.get("name"))
        
        
df1 = pd.DataFrame(b, columns = [(str_opt.replace("ga:","").upper()) for str_ in a for str_opt in str_])
df2 = pd.DataFrame(d, columns = [(sub.replace("ga:","").upper()) for sub in c])
final_df = pd.concat([df1, df2], axis = 1)
final_df

Saving credential.json to credential (21).json


,SOURCE,MEDIUM,SESSIONS,USERS,PAGEVIEWS,BOUNCERATE
0,(direct),(none),627,575,1023,65.23125996810208
1,[0212042],www.tinyurl.com/hgaco4fha3,1,1,1,100.0
2,[6324728],www.tinyurl.com/baufpn4th6j,1,1,1,100.0
3,10khits.com,referral,12,12,15,66.66666666666666
4,14.142.64.18:8080,referral,1,1,1,100.0
5,5-steps-to-start-business.com,referral,8,8,8,100.0
6,bing,organic,8,7,18,62.5
7,blackhatworld.com,referral,1,1,2,0.0
8,brainwizardphd.com,referral,1,1,1,100.0
9,complaint92797884.copyrightinstitute.org,referral,1,1,2,0.0
